Separate file to download posts in the background and save them to the folder so that I can work on the rest of the project while it is happening

I did this before we started looking into BeautifulSoup. I don't want to mess with it now. Legacy Code!

In [1]:
import time, requests
import pandas as pd

In [2]:
import pixiedust

Pixiedust database opened successfully


In [3]:
#Max num of pages to try and get
posts=50

In [4]:
# LFG':pd.DataFrame(), 'nosleep':pd.DataFrame(), 
jsons = {'talesfromtechsupport':pd.DataFrame(), 'buildapc':pd.DataFrame()}
the_headers = {'User-agent': 'project3 Bot 0.4'}

In [5]:
features = [
    'subreddit',
    'url',
    'author',
    'domain',
    'downs',
    'is_self', 
    'is_video', 
    'likes',
    'media',
    'num_comments',
    'num_crossposts',
    'num_reports',
    'selftext',
    'score',
    'title',
    'ups'
]

In [6]:
%%time

# The meat of this file.  It goes through and gets posts+1 pages of posts from each subreddit, gets out the relevant features, adds them to a DataFrame and appends to that DataFrame until the loop finishes
for k, v in jsons.items():
    URL = "http://www.reddit.com/r/" + k + ".json"

    res = requests.get(URL, headers = the_headers)
    the_json = res.json()

    pre_temp_df = pd.DataFrame(the_json['data']['children'])
    pre_temp_df = pd.DataFrame(list(pre_temp_df['data']))[features]
    jsons[k] = pre_temp_df.copy()
    
    #print('a', pre_temp_df.shape)
    
    URL_EXTENDER = "?after="

    print(k, 'Iterations:')
    for i in range(posts): 
        last_title = the_json['data']['after']

        try:
        #retrieve new data
            if last_title==None:
                break
                
            temp_data = requests.get(URL+URL_EXTENDER+last_title, headers = the_headers)

            the_json = temp_data.json()
            
            #add to the temp df more
        
            pre_temp_df = pd.DataFrame(the_json['data']['children'])

            if (len(pre_temp_df.columns) == 0):
                break
            pre_temp_df = pd.DataFrame(list(pre_temp_df['data']))[features]
            #print('b', pre_temp_df.shape)

            jsons[k] = jsons[k].append(pre_temp_df, ignore_index = True)
            
            time.sleep(3)
            if i % 10 == 0:
                print()
            print('%02d' % (i+1), end=" ")

        except _ as e:
            print()
            print(e, "Let's see what we've got!")
            break
    
    #add temp df to main df
    print()
    print(k,' temp df shape', jsons[k].shape)
    print('\n\n\n')
    
#print(jsons)

talesfromtechsupport Iterations:

01 02 03 04 05 06 07 08 09 10 
11 12 13 14 15 16 17 18 19 20 
21 22 23 24 25 26 27 28 29 30 
31 32 33 34 35 36 37 38 39 
talesfromtechsupport  temp df shape (979, 16)




buildapc Iterations:

01 02 03 04 05 06 07 08 09 10 
11 12 13 14 15 16 17 18 19 20 
21 22 23 24 25 26 27 28 29 30 
31 32 33 34 35 36 37 38 39 
buildapc  temp df shape (992, 16)




CPU times: user 2.36 s, sys: 157 ms, total: 2.51 s
Wall time: 4min 29s


In [7]:
print(last_title==None)

True


In [8]:
for k, v in jsons.items():
    print(v.shape)

(979, 16)
(992, 16)


In [9]:
main_df = pd.DataFrame()
for k, v in jsons.items():
    main_df = main_df.append(v.copy(), ignore_index = True)
main_df.drop_duplicates(inplace = True, subset=['url'])
print(main_df.shape)

(1971, 16)


In [10]:
#main_df.head()

In [11]:
#save to csv
path = './data/new_data.csv'
main_df.to_csv(path, index=False)

In [ ]:
#test the csv
path = './data/new_data.csv'
test_db = pd.read_csv(path)
print(test_db.shape)
test_db.equals(main_df) #returns false because the Nones are now NaNs


In [ ]:
test_db.head()